In [1]:
# 읽어올 파일 이름
file_name_train = "train_data.csv"

In [2]:
import pandas as pd
# 파일에 데이터가 탭으로 구분되어있음
train_data = pd.read_csv(file_name_train, encoding='utf-8-sig')

In [11]:
from soynlp.normalizer import repeat_normalize
# 중복 제거
remove_keywords = ['그닥', '빼고는', '빼곤', '말고는', '나쁘지', '지만']
pattern = '|'.join(remove_keywords)
filtered_data = train_data[~train_data['text'].str.contains(pattern, case=False, na=False)]

In [ ]:
print(f"원래 데이터 개수: {len(train_data)}")
print(f"필터링 후 데이터 개수: {len(filtered_data)}")

원래 데이터 개수: 20609
필터링 후 데이터 개수: 18348


In [12]:
label_counts = filtered_data['label'].value_counts().sort_index()

In [13]:
print(label_counts)

label
0    8758
1    9590
Name: count, dtype: int64


In [14]:
filtered_data.to_csv('0717_train_data.csv', encoding='utf-8-sig', index=False)

In [ ]:
# train_data.to_csv('중복제거한_train_data.csv', encoding='utf-8-sig', index=False)
# test_data.to_csv('중복제거한_test_data.csv', encoding='utf-8-sig', index=False)

In [47]:
new_df = pd.read_csv('0717_train_data.csv', encoding='utf-8-sig')

In [48]:
# 훈련용 리뷰 데이터 개수 출력
print(f"훈련용 리뷰 개수 : {len(new_df)}")
new_df['label'].value_counts()


훈련용 리뷰 개수 : 18348


label
1    9590
0    8758
Name: count, dtype: int64

In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# 텍스트와 라벨 나누기
texts = new_df['text'].tolist()
labels = new_df['label'].tolist()

# # 훈련/검증 분할 (예: 80% 훈련 / 20% 검증)
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

In [ ]:
# train_df, test_df = train_test_split(
#     new_df,
#     test_size=0.2,
#     stratify=new_df['label'],
#     random_state=42
# )

# # 5. 저장
# train_df.to_csv("saved_data/0715_train_data.csv", index=False, encoding="utf-8-sig")
# test_df.to_csv("saved_data/0715_test.csv", index=False, encoding="utf-8-sig")

In [51]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 토크나이저 준비
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# 시퀀스로 변환
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# 패딩
max_len = 70
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

# y는 리스트이므로 numpy로 변환
import numpy as np
y_train = np.array(y_train)
y_test = np.array(y_test)


In [52]:
import json

config = {
    'max_len': 70,
    'vocab_size': 10000
}

with open('saved_data/config.json', 'w') as f:
    json.dump(config, f)

In [53]:
import pickle
import numpy as np
# 저장
with open('saved_data/X_train_sequence.pkl', 'wb') as f:
    pickle.dump(X_train_pad, f)

with open('saved_data/y_train_sequence.pkl', 'wb') as f:
    pickle.dump(y_train, f)
    
with open('saved_data/X_test_sequence.pkl', 'wb') as f:
    pickle.dump(X_test_pad, f)

with open('saved_data/y_test_sequence.pkl', 'wb') as f:
    pickle.dump(y_test, f)


In [54]:
with open('saved_data/saved_tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [ ]:
y_train = [label_map[label] for label in y_train]
y_test = [label_map[label] for label in y_test]

In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=max_len),
    LSTM(64, return_sequences=False),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(
    X_train_pad, y_train,
    validation_data=(X_test_pad, y_test),
    epochs=10,
    batch_size=64
)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "c:\Users\MYCOM\.conda\envs\azen\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\MYCOM\.conda\envs\azen\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\MYCOM\.conda\envs\azen\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\MYCOM\.conda\envs\azen\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\MYCOM\.conda\envs\azen\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\MYCOM\.conda\envs\azen\lib\site-packages\tornado\platform\asyncio.py", line 211, in start
      self.asyncio_loop.run_

In [ ]:
import numpy as np
# 결측치 제거
train_data['text'] = train_data['text'].str.replace('[^a-zA-Z가-힣0-9 ]','',regex=True)
train_data['text'] = train_data['text'].str.replace('^ +','',regex=True)
train_data.dropna(how='any', inplace=True)

In [7]:
test_data['text'] = test_data['text'].str.replace('[^a-zA-Z가-힣0-9 ]','',regex=True)
test_data['text'] = test_data['text'].str.replace('^ +','',regex=True)
test_data.dropna(how='any', inplace=True)

In [8]:
# 중복, 결측치 제거 후 데이터 개수
print(f"전처리 후 훈련 데이터 개수 : {train_data.shape}")
print(f"전처리 후 테스트 데이터 개수 : {test_data.shape}")
# print(f"전처리 후 테스트 데이터 개수 : {y_review.shape}")

전처리 후 훈련 데이터 개수 : (20609, 5)
전처리 후 테스트 데이터 개수 : (5153, 5)


In [9]:
# 불용어 목록 선언
stopwords = list(set([
    '이', '가', '은', '는', '을', '를', '의', '에', '에서', '에게', '께', '로', '으로', 
    '와', '과', '보다', '처럼', '만큼', '같이', '까지', '마저', '조차', '부터', 
    '이나', '나', '이며', '며', '등', '하다', '한다', '하고', '하니', '하면', 
    '되어', '되다', '되고', '되니', '입니다', '습니다', 'ㅂ니다', '어요', '아요', '다',
    '고', '면', '게', '지', '죠',
    '그리고', '그러나', '하지만', '그런데', '그래서', '그러면', '그러므로', '따라서', 
    '또한', '또는', '및', '즉', '한편', '반면에', '근데',
    '나', '저', '우리', '저희', '너', '너희', '당신', '그', '그녀', '그들', '누구',
    '무엇', '어디', '언제', '어느', '이것', '그것', '저것', '여기', '거기', '저기', 
    '이쪽', '그쪽', '저쪽',
    '하나', '둘', '셋', '넷', '다섯', '여섯', '일곱', '여덟', '아홉', '열',
    '일', '이', '삼', '사', '오', '육', '칠', '팔', '구', '십', '백', '천', '만',
    '첫째', '둘째', '셋째',
    '바로', '때', '것', '수', '문제', '경우', '부분', '이다',
    '내용', '결과', '자체', '가지', '있다',
    '않았어요', '있었어요', '했어요', '했는데요', '있는데요', '합니다', '없다', '나다','생각하다',
    '했다', '같다', '네요','아니다', '용하다', '물이',
    '뿐', '대로', '만', '따름', '김에', '터',
    '아', '아이고', '아이구', '아하', '어', '그래', '응', '네', '예', '아니', '않다', '안되다','안',
    '가다', '오다', '주다', '말다', '나다', '받다', '알다', '모르다', '싶다', '생각하다', '들다'
]))

In [10]:
# 형태소 분석기 객체 생성
from konlpy.tag import Okt
okt  = Okt()

In [ ]:
# 작업의 진행 상황을 시각적으로 표시해주는 라이브러리
from tqdm import tqdm

In [12]:
X_train = []
y_train = []

for sentence, label in tqdm(zip(train_data['text'], train_data['label'])):
    tokens = okt.morphs(sentence)
    filtered = [w for w in tokens if w not in stopwords]
    if len(filtered) > 0:
        X_train.append(filtered)
        y_train.append(label)

0it [00:00, ?it/s]

20609it [01:38, 209.17it/s]


In [13]:
print(X_train[:20])
print(y_train[:20])

[['가격', '대비', '만족합니다', '잘', '쉬다', '갑니다'], ['전주', '시내', '가까운', '모텔', '가에', '인접', '해있고', '찾기', '쉬워요', '왠만한', '수도권', '호텔', '못', '않게', '인테리어', '청결', '도', '서비스', '깔끔하고', '완벽했습니다', '물', '수압', '도', '세고', '바닥', '벽면', '금방', '제', '습', '되서', '습하지', '않고', '에어컨', '담배', '냄새', '안나', '요', '오래된', '곳', '거진', '담배', '냄새', '베서', '다시', '전주', '방문', '하게', '되면', '재', '방문', '할거예요'], ['제', '여행', '기간', '중', '사용', '했던', '게스트하우스', '중', '제일', '좋았습니다', '위치', '가성', '비는', '말', '할', '도', '없었구요', '솔직히', '야', '놀자', '리뷰', '보고', '걱정', '했었는데', '정말', '잘', '쉬', '갑니다'], ['속', '터미널', '가까워서', '이동', '할', '편안했어요', '방도', '깨끗하고', '난방', '잘', '되어서', '추운', '날씨', '따뜻하게', '잘', '보냈습니다', '다음', '전주', '오게', '되면', '또', '이용', '싶어요'], ['터미널', '이랑', '가까워서', '좋았지만', '화장실', '전기', '도', '들어오고', '휴대폰', '충전기', '도', '고장', '나서', '숙박', '하는데', '불편했어요'], ['넷플방', '결제', '했는데', '넷플방', '돈', '더', '줘', '야한다고', '처음', '결제', '할', '가격', '다르게', '하셨으면', '걸', '결제', '했는데'], ['침대', '스프링', '꺼져있네요'], ['아주'], ['아조', '조', '타', '야'], ['베개', '꿉', '꿉한', '냄새', '났어요'], ['사장', '님',

In [14]:
X_test = []
y_test = []

for sentence, label in tqdm(zip(test_data['text'], test_data['label'])):
    tokens = okt.morphs(sentence)
    filtered = [w for w in tokens if w not in stopwords]
    if len(filtered) > 0:
        X_test.append(filtered)
        y_test.append(label)

5153it [00:26, 195.53it/s]


In [15]:
y_train = np.array(train_data['label'], dtype=int)
y_test = np.array(test_data['label'], dtype=int)

In [16]:
import pickle
with open('LSTM/X_train.pickle', 'wb') as f:
    pickle.dump(X_train, f)
with open('LSTM/X_test.pickle', 'wb') as f:
    pickle.dump(X_test, f)
with open('LSTM/y_train.pickle', 'wb') as f:
    pickle.dump(y_train, f)
with open('LSTM/y_test.pickle', 'wb') as f:
    pickle.dump(y_test, f)

In [17]:
print(X_train[:10])
print(X_test[:10])
print(y_train[:10])
print(y_test[:10])

[['가격', '대비', '만족합니다', '잘', '쉬다', '갑니다'], ['전주', '시내', '가까운', '모텔', '가에', '인접', '해있고', '찾기', '쉬워요', '왠만한', '수도권', '호텔', '못', '않게', '인테리어', '청결', '도', '서비스', '깔끔하고', '완벽했습니다', '물', '수압', '도', '세고', '바닥', '벽면', '금방', '제', '습', '되서', '습하지', '않고', '에어컨', '담배', '냄새', '안나', '요', '오래된', '곳', '거진', '담배', '냄새', '베서', '다시', '전주', '방문', '하게', '되면', '재', '방문', '할거예요'], ['제', '여행', '기간', '중', '사용', '했던', '게스트하우스', '중', '제일', '좋았습니다', '위치', '가성', '비는', '말', '할', '도', '없었구요', '솔직히', '야', '놀자', '리뷰', '보고', '걱정', '했었는데', '정말', '잘', '쉬', '갑니다'], ['속', '터미널', '가까워서', '이동', '할', '편안했어요', '방도', '깨끗하고', '난방', '잘', '되어서', '추운', '날씨', '따뜻하게', '잘', '보냈습니다', '다음', '전주', '오게', '되면', '또', '이용', '싶어요'], ['터미널', '이랑', '가까워서', '좋았지만', '화장실', '전기', '도', '들어오고', '휴대폰', '충전기', '도', '고장', '나서', '숙박', '하는데', '불편했어요'], ['넷플방', '결제', '했는데', '넷플방', '돈', '더', '줘', '야한다고', '처음', '결제', '할', '가격', '다르게', '하셨으면', '걸', '결제', '했는데'], ['침대', '스프링', '꺼져있네요'], ['아주'], ['아조', '조', '타', '야'], ['베개', '꿉', '꿉한', '냄새', '났어요']]
[['전북대', '병